In [2]:
# 필요한 패키지 설치 (Colab 환경에서는 필요 시 주석 해제)
# !pip install sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
with open("translated_menu_structured.json", "r", encoding="utf-8") as f:
    menu_database = json.load(f)

In [4]:
def extract_menu_info(korean_menu_name):
    for item in menu_database:
        if item["menu_name"]["ko"] == korean_menu_name:
            return {
                "menu_name_translations": {
                    "en": item["menu_name"].get("en", ""),
                    "zh": item["menu_name"].get("zh", ""),
                    "ja": item["menu_name"].get("ja", "")
                },
                "ingredients": item["ingredients"].get("ko", "")
            }
    return None

In [5]:
# 다국어 지원 S-BERT 모델 로드 (예: paraphrase-multilingual-mpnet-base-v2)
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# 전처리된 텍스트 리스트 추출
texts = [item["menu_name"]["ko"] + ": " + item["ingredients"]["ko"] for item in menu_database]

# S-BERT를 통해 임베딩 생성 (대량 데이터의 경우 배치 사이즈 조절)
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True)
embeddings = np.array(embeddings)

print("임베딩 생성 완료, 임베딩 shape:", embeddings.shape)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/860 [00:00<?, ?it/s]

임베딩 생성 완료, 임베딩 shape: (55019, 768)


In [6]:
def get_user_profile_embedding(favorite_foods):
    """
    favorite_foods: 사용자가 좋아하는 음식 리스트 (예: ["비빔밥", "김치찌개", ...])
    model: SentenceTransformer 모델
    반환: 평균 사용자 프로필 임베딩 (numpy array)
    """
    food_infos = []
    for food in favorite_foods:
        food_info = extract_menu_info(food)
        if food_info:
            food_infos.append(food + ": " + food_info["ingredients"])
        else:
            food_infos.append(food)
    user_embeddings = model.encode(food_infos)
    user_profile = np.mean(user_embeddings, axis=0)
    return user_profile

def recommend_menu_from_scanned_menu(user_favorites, menu_items, top_n=5):
    """
    user_favorites: 사용자의 선호 음식 리스트 (음식 이름+재료 결합 텍스트, 예: ["비빔밥", "김치찌개", ...])
    menu_items: 백엔드에서 받아온 메뉴판 데이터 (예: ["불고기", "치킨", ...])
    top_n: 추천할 메뉴 수

    반환: 사용자의 선호 클러스터와 해당 클러스터 내에서 코사인 유사도 기준으로 정렬된 추천 메뉴 리스트 (메뉴 텍스트와 유사도 포함)
    """
    # 1. 사용자 선호 음식으로부터 사용자 프로필 임베딩 생성
    user_profile = get_user_profile_embedding(user_favorites)

    # 2. 메뉴판 데이터에 대해 S-BERT 임베딩 생성
    menu_infos = []
    for menu_item in menu_items:
        menu_info = extract_menu_info(menu_item)
        if menu_info:
            menu_infos.append(menu_item + ": " + menu_info["ingredients"])
    if not menu_infos: # 빈 리스트가 될 경우 raise error
        raise Exception("메뉴판이 제대로 인식되지 않았거나 데이터셋에 포함된 음식이 없습니다.")
    menu_embeddings = model.encode(menu_items)
    menu_embeddings = np.array(menu_embeddings)

    # 3. 메뉴 항목과 사용자 프로필 간 코사인 유사도 계산
    similarities = cosine_similarity([user_profile], menu_embeddings)[0]

    # 4. 유사도 기준 내림차순 정렬하여 상위 top_n 메뉴 선택
    sorted_idx = np.argsort(similarities)[::-1]

    # 5. 추천 결과 출력: 메뉴 텍스트와 해당 유사도 점수
    recommendations = []
    cnt, i = 0, 0
    while cnt < top_n:
        # 알러지 필터링은 나중에 구현
        recommendations.append((menu_items[sorted_idx[i]], similarities[sorted_idx[i]]))
        i += 1
        if i >= len(menu_items):
            break
        cnt += 1  # 알러지 필터링 할 때는 따로 빼서 할 예정

    return recommendations

# 예시 실행:
# 사용자 선호 음식
user_favorites = ["비빔밥", "김치찌개", "된장찌개", "불고기", "잡채"]

# 메뉴판 데이터 (더미 데이터 예시)
dummy_menu_items = [
    "불고기", "비빔밥", "김치찌개", "된장찌개", "잡채", "갈비탕", "순두부찌개", "떡볶이", "비빔냉면",
    "삼겹살", "냉면", "제육볶음", "콩나물국", "치킨", "피자", "파스타", "샐러드", "스테이크", "라멘", "초밥"
]


recommendations = recommend_menu_from_scanned_menu(user_favorites, dummy_menu_items, top_n=5)

print("추천 결과:")
for item, score in recommendations:
    print(f"{item} (추천도: {score:.3f})")


추천 결과:
제육볶음 (추천도: 0.725)
떡볶이 (추천도: 0.654)
비빔밥 (추천도: 0.631)
삼겹살 (추천도: 0.617)
샐러드 (추천도: 0.616)


In [7]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=55180c53edec08970047f671d8aec848a24e0a0465d36dd9f008dee3ad183d13
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [12]:
from googletrans import Translator

translator = Translator()

langs = {"중국어": "zh-cn", "일본어": "ja", "영어": "en"}
user_lang = "영어"

for item, _ in recommendations:
    if user_lang == "한국어":
        print(item)
    else:
        print(translator.translate(item, dest=langs[user_lang]).text)

Stir -fried
tteokbokki
Bibimbap
pork belly
salad
